In [21]:
# import dependencies 
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
 
from bs4 import BeautifulSoup as bs
import requests as req

from sqlalchemy import create_engine

In [2]:
# Request beer consumption data from link
link_request = req.get("https://www.thrillist.com/drink/nation/beer-consumption-by-state/3973432")

# Create data soup using BeautifulSoup
data_soup = bs(link_request.text, 'html.parser')

# Scrape data using class tag as below
scrape = data_soup.find_all('p', attrs={'class':'body-text__paragraph-text font--body has-spacing'})

# Preview Data
data = scrape[1]
data

<p class="body-text__paragraph-text font--body has-spacing"><strong>Alabama: </strong>30.2 gallons<br/><strong>Alaska: </strong>28.7 gallons<br/><strong>Arizona: </strong>29.5 gallons<br/><strong>Arkansas: </strong>26.7 gallons<br/><strong>California: </strong>25.5 gallons<br/><strong>Colorado: </strong>30 gallons<br/><strong>Connecticut: </strong>22 gallons<br/><strong>Delaware: </strong>33.6 gallons<br/><strong>Florida: </strong>27.4 gallons<br/><strong>Georgia: </strong>25.7 gallons<br/><strong>Hawaii: </strong>30.4 gallons<br/><strong>Idaho: </strong>27.8 gallons<br/><strong>Illinois: </strong>29.1 gallons<br/><strong>Indiana: </strong>25.9 gallons<br/><strong>Iowa: </strong>33.6 gallons<br/><strong>Kansas: </strong>28.3 gallons<br/><strong>Kentucky: </strong>24.4 gallons<br/><strong>Louisiana: </strong>33.9 gallons<br/><strong>Maine: </strong>34 gallons<br/><strong>Maryland: </strong>23.2 gallons<br/><strong>Massachusetts: </strong>26.2 gallons<br/><strong>Michigan: </strong>26.8 

In [3]:
# Change data into list to begin formatting data
data_list = []
for x in data: 
    data_list.append(x)
    
# Preview list 
# data_list[0], data_list[1], data_list[2], 
    
# Create 2 lists. One with states and another with gallons consumed
states = []
gallons = []
for x in data_list:
    if str("gallons") in str(x):
        gallons.append(x)
    if str("</strong>")in str(x):
        states.append(x)
    else: pass
    

In [4]:
# Preview state list
states[0]

# Preview gallons list
#gallons[0]

<strong>Alabama: </strong>

In [5]:
# Extract state string from state list
state_list = []
for x in states:
    # Multiple splits to extract the states
    state1 = str(x).split(": <")[0]
    state2 = str(state1).split(">")[1]    
    state_list.append(state2)

state_list[0]

'Alabama'

In [6]:
# Extract gallon number from gallons list
gallons_list = []
for x in gallons:
    gallon1 = str(x).split(" ")[0]
    gallons_list.append(gallon1)
    
gallons_list[0]

'30.2'

In [15]:
# Create beer consumption per state DataFrame
beer_consumption_df = pd.DataFrame({"state": state_list,
                                   "consumption_in_gallons": gallons_list})
# Preview beer consumption DataFrame
beer_consumption_df.head()

,state,consumption_in_gallons
0,Alabama,30.2
1,Alaska,28.7
2,Arizona,29.5
3,Arkansas,26.7
4,California,25.5


In [16]:
# Create state DataFrame with state's abbreviation
states_url = 'http://app02.clerk.org/menu/ccis/Help/CCIS%20Codes/state_codes.html'
tables = pd.read_html(states_url)
states_table = tables[0]
states_table_df = states_table.rename(columns={'Code': 'state_abbreviation',
                                                'Description': 'state'})

# Preview state DataFrame
states_table_df.head()

,state_abbreviation,state
0,AA,Armed Forces (the) Americas
1,AB,Alberta
2,AE,Armed Forces Europe
3,AK,Alaska
4,AL,Alabama


In [17]:
# Merge beerconsumption and  and states df
merged_states_df = pd.merge(beer_consumption_df, states_table_df , on="state", how="inner")
merged_states_df.head()

,state,consumption_in_gallons,state_abbreviation
0,Alabama,30.2,AL
1,Alaska,28.7,AK
2,Arizona,29.5,AZ
3,Arkansas,26.7,AR
4,California,25.5,CA


In [18]:
# Reset Index
indexed_beer_consumption_df = merged_states_df.reset_index()
indexed_beer_consumption_df.head()

,index,state,consumption_in_gallons,state_abbreviation
0,0,Alabama,30.2,AL
1,1,Alaska,28.7,AK
2,2,Arizona,29.5,AZ
3,3,Arkansas,26.7,AR
4,4,California,25.5,CA


In [19]:
#save as new csv
indexed_beer_consumption_df.to_csv('beer_consumption_df.csv')

# Connect to local database

In [22]:
rds_connection_string = "postgres:postgres@localhost:5432/beers_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Create new data (parking_fines) with select columns

In [23]:
beer_consumption = indexed_beer_consumption_df[['index', 'state', 'consumption_in_gallons', 'state_abbreviation']].copy()
beer_consumption.reset_index(drop=True)

,index,state,consumption_in_gallons,state_abbreviation
0,0,Alabama,30.2,AL
1,1,Alaska,28.7,AK
2,2,Arizona,29.5,AZ
3,3,Arkansas,26.7,AR
4,4,California,25.5,CA
5,5,Colorado,30,CO
6,6,Connecticut,22,CT
7,7,Delaware,33.6,DE
8,8,Florida,27.4,FL
9,9,Georgia,25.7,GA


Make sure the above three tables are created in postgres as well before you start the next steps

### Check for tables

In [24]:
engine.table_names()

['final_beer', 'beer_consumption']

### Use pandas to load csv converted DataFrame into database

In [25]:
beer_consumption.to_sql(name='beer_consumption', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the customer_name table
* NOTE: can also check using pgAdmin

In [26]:
pd.read_sql_query('select * from beer_consumption', con=engine).head()

,index,state,consumption_in_gallons,state_abbreviation
0,0,Alabama,30.2,AL
1,1,Alaska,28.7,AK
2,2,Arizona,29.5,AZ
3,3,Arkansas,26.7,AR
4,4,California,25.5,CA
